# Homework 6

# Arthur Setiawan 11-2-2023

In [2]:
import altair as alt
import pandas as pd
import numpy as np

#Disable maxrows check
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Dataset Background

The dataset chosen comes from the U.S. Department of Energy, specifically looking at numerical variables of emissions and fuel efficiency of vehicles that were released in the U.S. from 1984 - 2023. Given multiple state governments like in Washington and California are banning the sale of non-electric cars from 2035 onwards, we are interested in exploring how the rapid rise of electric vehicles introduced to the market will have impact on better EPA green house gas scores, reduced petroleum consumption, and reduced CO2 output as well. Furthermore, I am interested in exploring which makes and vehicle models along with their properties and characteristics like transmission, EPA vehicle class, and volumetric characteristics tend to induce worse environmental impacts or lower/higher fuel economy ratings, allowing me to identify which makes will need to step up their game to follow up with the mandated sale of emissions free vehicles by 2035. 

There are many more variables in the dataset, but I have selected the selected 33 variables, including unique identifier of ID, and important identifying variables like year, make, model, transmission, cylinder, alternative fuel type, vehicle class, displacement, and drivetrain. The quantitative, categorical, and boolean variables we have are luggage and passenger volume, startstop technology, gas-guzzler identifier, fuel economy score, fuel type, hybrid identifier, combined mpg, annual petrol consumption in barrels, annual fuel cost, hybrid mpg, co2 output, greenhouse gas score, electricity consumption, range, electric motor type, and time needed to charge electric vehicle

Link to data source and documentation: https://www.fueleconomy.gov/feg/ws/index.shtml#fuelType1

In [3]:
#Taking dataset from U.S. Department of Energy, includes emissions, fuel types, fuel efficiency,
# luggage and passenger volume, make, model, etc... data for cars made between 1984 - 2024
dataurl = 'vehicles.csv'

#Reorder cols, and choose which cols we want to include
desired_order =    ['id',
                   'year',
                   'make',
                   'model',
                   'trany',
                   'cylinders',
                   'atvType', #type of alternative fuel or advanced technology vehicle
                   'VClass', #epa vehicle class
                   'displ', #engine displacement in litres
                   'drive', #drivetrain
                   'hlv', # luggage volume (cubic feet)
                   'hpv', # passenger volume (cubic feet)
                   'startStop', #StartStop tech
                   'guzzler', #gas guzzler identifier
                   'feScore', #epa fuel economy score
                   'fuelType', # fuel type with fuelType1 and fuelType2 
                   'phevBlended', #if true, this vehicle operates on a blend of gasoline and electricity in charge depleting mode
                   'comb08', #comb08 - combined MPG for fuelType1 (2)
                   'barrels08', #annual petroleum consumption in barrels for fuelType1
                   'fuelCost08', #annual fuel cost for fuelType1
                   'phevComb', #EPA composite gasoline-electricity combined city-highway MPGe for plug-in hybrid vehicles
                   'co2', #tailpipe CO2 in grams/mile fuelType1
                   'ghgScore', #EPA greenhousegas score fueltype1
                    
                   #Electeric-only variables
                   'combE', #combined electricity consumption in kwhrs/100miles
                   'rangeA', #EPA range for fuelType2
                   'evMotor', #electric motor (kw-hrs)
                   'charge120', #time to charge electric vehicle in hrs at 120V
                   'charge240'] #time to charge electric vehicle in hrs at 240V

#Read URL 
auto = pd.read_csv(dataurl, na_values='*',
                  usecols=desired_order)

#Reorder accordingly
auto = auto[desired_order]

#Column renames
new_colnames = {'atvType':'atv_type', 
                'VClass':'epa_vclass',
                'hlv':'vol_luggage',
                'hpv':'vol_passenger',
                'startStop':'start_stop',
                'feScore':'fe_score',
                'fuelType': 'fuel_type',
                'phevBlended':'phev_blended',
                'comb08':'combmpg',
                'barrels08': 'annual_barrel',
                'fuelCost08': 'fuelcost',
                'phevComb': 'combmpg_phev',
                'co2': 'co2',
                'ghgScore': 'ghgscore',
                'combE': 'comb_electric',
                'rangeA': 'range_ft2',
                'evMotor': 'evmotor',
                'charge120': 'hr_charge120v',
                'charge240': 'hr_charge240v'}

#Rename cols for better interpratbility
auto.rename(columns=new_colnames, inplace=True)

#Replace nulls in atv_type to ICE cars
auto['atv_type'].fillna('ICE', inplace=True)

#Glimpse
auto.head()

C:\Users\setia\AppData\Local\Temp\ipykernel_31124\2311313322.py:38: DtypeWarning: Columns (72,73,80) have mixed types. Specify dtype option on import or set low_memory=False.
  auto = pd.read_csv(dataurl, na_values='*',


id  year        make                model            trany  cylinders  \
0      1  1985  Alfa Romeo   Spider Veloce 2000     Manual 5-spd        4.0   
1     10  1985     Ferrari           Testarossa     Manual 5-spd       12.0   
2    100  1985       Dodge              Charger     Manual 5-spd        4.0   
3   1000  1985       Dodge  B150/B250 Wagon 2WD  Automatic 3-spd        8.0   
4  10000  1993      Subaru     Legacy AWD Turbo     Manual 5-spd        4.0   

  atv_type       epa_vclass  displ                       drive  ...  \
0      ICE      Two Seaters    2.0            Rear-Wheel Drive  ...   
1      ICE      Two Seaters    4.9            Rear-Wheel Drive  ...   
2      ICE  Subcompact Cars    2.2           Front-Wheel Drive  ...   
3      ICE             Vans    5.2            Rear-Wheel Drive  ...   
4      ICE     Compact Cars    2.2  4-Wheel or All-Wheel Drive  ...   

   annual_barrel  fuelcost combmpg_phev co2  ghgscore comb_electric  \
0      14.167143      2750            0  -1        -1           0.0   
1      27.046364      5300            0  -1        -1           0.0   
2      11.018889      2150            0  -1        -1           0.0   
3      27.046364      5300            0  -1        -1           0.0   
4      15.658421      3750            0  -1        -1           0.0   

   range_ft2  evmotor  hr_charge120v  hr_charge240v  
0        NaN      NaN            0.0            0.0  
1        NaN      NaN            0.0            0.0  
2        NaN      NaN            0.0            0.0  
3        NaN      NaN            0.0            0.0  
4        NaN      NaN            0.0            0.0  

[5 rows x 28 columns]

In [172]:
#Recategorize make into manufacturing groups, based on country of origin

combined_make_to_grp = {
    ('Volkswagen', 'Audi', 'Porsche', 'Mercedes-Benz', 'BMW', 'RUF Automobile', 'Ruf Automobile Gmbh', 'smart', 'BMW Alpina',): 'German',
    ('Chevrolet', 'Lincoln', 'Ford', 'Buick', 'Cadillac', 'Dodge', 'GMC', 'Roush Performance',
     'Jeep', 'Chrysler', 'Ram', 'SRT', 'Fisker', 'Karma', 'VPG', 'Mobility Ventures LLC',
     'Saturn', 'AM General', 'Geo', 'American Motors Corporation', 'Saleen', 'Vector',
     'Fisker', 'Tesla', 'Shelby', 'Saleen Performance', 'Rivian', 'Lucid', 'Vinfast', 'Karma'): 'American',
    ('Nissan', 'Lexus', 'Mazda', 'Scion', 'Toyota', 'Subaru', 
     'Infiniti', 'Acura', 'Suzuki', 'Honda', 'STI', 'Mitsubishi', 'Isuzu'): 'Japanese',
    ('Volvo', 'Polestar', 'Saab'): 'Swedish',
    ('Hyundai', 'Kia', 'Genesis', 'Daewoo'): 'Korean',
    ('Bentley', 'Land Rover', 'Rolls-Royce', 'Jaguar', 'MINI', 'Lotus', 'Aston Martin', 'McLaren Automotive',
     'Morgan'): 'British',
    ('Ferrari', 'Lamborghini', 'Alfa Romeo', 'Fiat', 'Maserati', 'Koenigsegg', 'Pininfarina', 'Bertone'): 'Italian',
    ('Pagani', 'Koenigsegg', 'Bugatti', 'TVR Engineering Ltd', 'Spyker'): 'Specialty Market',
    ('Eagle', 'E. P. Dutton, Inc.', 'PAS, Inc',  'Panoz Auto-Development',  'Dabryan Coach Builders Inc', 'Federal Coach', 'Qvale',  'Autokraft Limited', 'Panther Car Company Limited', 'Texas Coach Company', 
     'Yugo',  'Renault', 'Bitter Gmbh and Co. Kg', 'Lambda Control Systems', 'Merkur', 'Peugeot',  'London Coach Co Inc',
     'Tecstar, LP', 'Kenyon Corporation Of America', 'Avanti Motor Corporation', 'Bill Dovell Motor Car Company', 'Import Foreign Auto Sales Inc', 
     'S and S Coach Company  E.p. Dutton', 'Superior Coaches Div E.p. Dutton', 'Vixen Motor Company', 'Volga Associated Automobile', 'Wallace Environmental', 'Import Trade Services', 'J.K. Motors', 
     'Panos', 'Quantum Technologies', 'London Taxi', 'Red Shift Ltd.',  'Excalibur Autos', 'Mahindra', 'Azure Dynamics', 'CODA Automotive', 
     'Mcevoy Motors', 'BYD', 'ASC Incorporated', 'CCC Engineering', 'Aurora Cars Ltd', 'Dacia', 'Kandi',  'JBA Motorcars, Inc.',
     'Lordstown', 'Grumman Allied Industries', 'Environmental Rsch and Devp Corp', 'Evans Automobiles', 'Laforza Automobile Inc', 'Consulier Industries Inc', 'Goldacre', 
     'Isis Imports Ltd', 'PAS Inc - GMC'): 'Other Markets'
}

#Apply dictionary to new column that we call 'manuf_grp'
auto['manuf_grp'] = auto['make'].apply(lambda make: next((v for k, v in combined_make_to_grp.items() if make in k), None))

#Recategorize vehicle class to minimize

combined_vehicle_class = {
    ('Compact Cars'): 'Compact Cars',
    ('Subcompact Cars', 'Minicompact Cars'): 'Subcompact Cars',
    ('Midsize Cars'): 'Midsize Cars',
    ('Large Cars'): 'Large Cars',
    ('Two Seaters'): 'Coupes',
    ('Standard Pickup Trucks', 'Standard Pickup Trucks 2WD', 'Standard Pickup Trucks 4WD',
     'Small Pickup Trucks', 'Small Pickup Trucks 2WD', 'Standard Pickup Trucks/2wd',
     'Small Pickup Trucks 4WD', ): 'Pickup Trucks',
    ('Sport Utility Vehicle - 4WD', 'Small Sport Utility Vehicle 4WD', 'Sport Utility Vehicle - 2WD',
     'Standard Sport Utility Vehicle 4WD', 'Small Sport Utility Vehicle 4WD', 'Small Sport Utility Vehicle 2WD',
     'Small Sport Utility Vehicle 2WD', 'Standard Sport Utility Vehicle 2WD'): 'SUVs',
    ('Vans', 'Vans, Cargo Type', 'Minivan - 2WD', 'Vans, Passenger Type', 
     'Minivan - 4WD', 'Vans Passenger'): 'Vans',
    ('Small Station Wagons', 'Midsize-Large Station Wagons', 'Midsize Station Wagons'): 'Station Wagons',
    ('Special Purpose Vehicles', 'Special Purpose Vehicle 2WD', 'Special Purpose Vehicle 4WD',
     'Special Purpose Vehicles/2wd', 'Special Purpose Vehicles/4wd', 'Special Purpose Vehicle'): 'Special Purpose Vehicles'
}

#Apply dictionary to new column that we call 'manuf_grp'
auto['vehicle_class'] = auto['epa_vclass'].apply(lambda make: next((v for k, v in combined_vehicle_class.items() if make in k), None))

#Take a smaller sample of the dataset
sample_auto = auto.sample(n=3000)

## Updated Plot with Interaction

The idea here is to use the desired vehicle class as a slider to recommend potential options for cars in specific years!

In [188]:
# Brush on vehicle class and MPG as first selection
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
)

#Class selection if clicked on a specific vehicle class
class_selection = alt.selection_multi(fields=['vehicle_class'], empty='all', on='click')

#Define arrays for selection
# vclass = sample_auto['vehicle_class'].unique().tolist()
# ftype = sample_auto['atv_type'].unique().tolist()


# single-value selection over [Major_Genre, MPAA_Rating] pairs
radio_drop = alt.selection_point(
    name='Choose',
    fields=['vehicle_class', 'atv_type'],
    bind={'Vehicle_Class': alt.binding_select(options=vclass), 'Fuel_Type': alt.binding_radio(options=ftype)}
)


plot_1 = alt.layer(
    
    #Main scatter chart showing cars based on their model years and annual fuel cost
    alt.Chart(sample_auto).mark_circle(size=20, clip=True).encode(
    alt.X('year:O', scale=alt.Scale(type='band'), title='Year', axis=alt.Axis(values=list(range(2000,2026,3)))),
    alt.Y('fuelcost:Q', title='Annual Fuel Cost', axis=alt.Axis(orient='left', grid=True)),
    alt.Color(value='#7D3C98'),
    opacity=alt.condition(class_selection, alt.value(1), alt.value(0.05)),
    tooltip=['year:N', 'make:N', 'model:N', 'vehicle_class:N', 'cylinders:Q', 'fuelcost:Q'])
).properties(width=250, height=250).facet(
    'manuf_grp:N',spacing=20, columns=3
    ).transform_filter('datum.manuf_grp != null').transform_filter((alt.datum.year > 2000
    ) & (alt.datum.manuf_grp != "Other Markets") & (alt.datum.manuf_grp != "Specialty Market"
    ) & (alt.datum.manuf_grp != "Swedish")).properties(title=alt.TitleParams(text="Step 2: Choose Your Desired Manufacturer Origin! Hover Over Point for Vehicle Information", fontSize=14, anchor='start', offset=20))

#Plot 2 We look at the counts of most popular vehicle class segments and their respective MPGs

class_bar =  alt.Chart(sample_auto).mark_bar(clip=True).encode(
  alt.X('combmpg:Q', title='Combined MPG', scale = alt.Scale(domain=[0,40])),
  alt.Y('count():Q'),
  alt.Color('vehicle_class:N', scale=alt.Scale(
    domain=['Subcompact Cars', 'Compact Cars', 'Midsize Cars', 'Station Wagons',  'SUVs', 'Pickup Trucks', 'Vans'],
    range=['#2ECC40', '#A0DF67', '#3D9970', '#0074D9', '#01A2D9', '#FF6600', '#FFA500']
  ), legend=None))

#Add a rule showing average MPG for each class segment
class_rule = alt.Chart(sample_auto).mark_rule(color='firebrick').encode(
  alt.X('combmpg:Q', aggregate='average', type='quantitative'))   


#Combined histogram of counts based on MPG faceted by vehicle segments along w/ average line
plot_2 = alt.layer(class_bar, class_rule).transform_filter(
                   (alt.datum.vehicle_class != "Coupes") &
                   (alt.datum.vehicle_class != "Large Cars") &
                   (alt.datum.vehicle_class != "Special Purpose Vehicles") 
                  ).properties(width=100,height=100).facet(
  column=alt.Column('vehicle_class:N', sort=['Subcompact Cars', 'Compact Cars', 'Midsize Cars', 
                                             'Station Wagons',  'SUVs',  'Pickup Trucks', 'Vans']
)).properties(title=alt.TitleParams(text="Step 1: Choose Your Desired Vehicle Class based on MPG! Click on Class Graph to Filter Recommended Vehicles", fontSize=14, anchor='start', offset=20)
             ).add_selection(class_selection)

#Descriptor for Step 2
desc_1= alt.Chart(pd.DataFrame({'text': ['NOTE [2]: There is a larger tendency for cars of German, British, and Italian origin to be gas guzzlers than American, Japanese, and Korean cars.']}
                              )).mark_text(
    align='left',
    baseline='top',
    fontSize=13,
    dx=0, 
    dy=10 
).encode(
    text='text'
)

#Descriptor for Step 1
desc_2 = alt.Chart(
    pd.DataFrame({'text': ['NOTE [1]: Vehicle Classes with lower combined MPGs (skewing left) are less fuel efficient, costing more to run while increasing your carbon footprint.']}
                )).mark_text(
    align='left',
    baseline='top',
    fontSize=13,
    dx=0,
    dy=10
).encode(
    text='text'
)

#Vertically Concatenate to Show finalized Plot
alt.vconcat(
plot_2, desc_2, plot_1, desc_1,
title="New Car Recommendation Dashboard",
).configure_header(title=None, labelFontSize=12, orient='top'
).configure_axisX(labelAngle=0, labelFontSize=12, gridOpacity=0.5
).configure_axisY(labelAngle=0, labelFontSize=12, gridOpacity=0.5
).configure_title(fontSize=20, anchor='middle', offset=15
)

alt.VConcatChart(...)

The basic encodings used in Homework 5 were revised here to adhere to newer and hopefully improved functionality of the dashboard. Now for the step 2, instead of it being a line graph with an area chart. It is now a scatter plot showing data points of each vehicle released year-by-year. We sampled this dataset to allow for a better-looking graph. This scatter plot encoding allows me to include a tooltip, which will allow the user to see which vehicles based on filtering will be best for their use case. Furthermore, this scatter plot encoding distinguishes data points from each other, allowing users to hover and have free choice on what they can look at, which we did not have any functionality prior. This tooltip includes the hovered over vehicle year, make, model, vehicle class, cylinders, and fuel cost, and enhances the plot because it helps the user to better understand what each car model name would be and provides a next avenue of research for them.

Next, the interaction I used is a multi selection on the vehicle class histograms at the top. The intention was to allow users to pick a vehicle class that they were interested in purchashing, and in doing so the graph auto highlights the specific models available for that vehicle class in our faceted graph. I then attempted to add a brush and a dropdown + radio button for further usability, but realized that in doing so sometimes removed the functionality of other selection methods, we see that adding more complexity is a definite limitation to creating dashboards on altair. Doing this allows the full interaction capabilites of a dashboard without it actually being a dashboard in PowerBI or Tableau. 